
##### refunder stream

this notebook starts a stream to score completed orders for potential refunds

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
#### TODO: give this job a schedule every 10min, skip if already running (no parallel runs)
#### TODO: for some reason the experiments tracing page is not registering the calls made through this, investigate that

In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.jobs as j
import os, json

w = WorkspaceClient()

# Resolve the workspace notebook path (keeps your approach)
notebook_abs_path = os.path.abspath("../jobs/refund_recommender_stream")
notebook_dbx_path = notebook_abs_path.replace(
    os.environ.get("DATABRICKS_WORKSPACE_ROOT", "/Workspace"),
    "/Workspace"
)

state_dir = "../.state"
os.makedirs(state_dir, exist_ok=True)
jobs_file_path = os.path.join(state_dir, "jobs")

job = w.jobs.create(
    name="Refund Recommender Stream",
    tasks=[
        j.Task(
            task_key="refund_recommender_stream",
            notebook_task=j.NotebookTask(
                notebook_path=notebook_dbx_path,
                base_parameters={
                    "CATALOG": dbutils.widgets.get("CATALOG"),
                    "REFUND_AGENT_ENDPOINT_NAME": dbutils.widgets.get("REFUND_AGENT_ENDPOINT_NAME")
                },
            )
        )
    ],
)
print(f"Created job_id={job.job_id}")
w.jobs.run_now(job_id=job.job_id)
with open(jobs_file_path, "a") as f:
    f.write(f"{job.job_id}\n")